
# --------------------------------------------

# 📘 **1 — Project Introduction**

# --------------------------------------------

This project implements **Brain Tumor Segmentation** using a **U-Net deep learning model** trained on MRI images.

### 🔍 What this system can do:

* Upload MRI Image (.png, .jpg, .jpeg)
* Upload MRI H5 file (.h5)
* Auto-convert H5 → PNG
* Run Deep Learning segmentation
* Generate:

  * ✔ Tumor Overlay Mask
  * ✔ Binary Mask (green tumor region)
  * ✔ Segmented Visualization

### 🌐 Web Deployment

The project also includes a **Flask Web App** with:

* Upload page
* Real-time segmentation results
* ngrok for Public Sharing

---

# --------------------------------------------

# 📘 **2 — Install All Dependencies**

# --------------------------------------------

This step installs all required libraries for:

* Deep Learning (PyTorch)
* Image Processing
* Flask Web Backend
* ngrok Deployment
* Albumentations Transforms
* MRI (.h5) file support

---

## ✅ **CELL 1 — Install Dependencies**




###Kaggle Code

In [ ]:
# Cell 1 — clean imports (NO ALBUMENTATIONS, NO SKLEARN)

import os, random, glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# manual split (no sklearn needed)
def simple_split(data_list, val_ratio=0.15, seed=42):
    random.Random(seed).shuffle(data_list)
    n_val = int(len(data_list) * val_ratio)
    val = data_list[:n_val]
    train = data_list[n_val:]
    return train, val

# PyTorch transforms instead of albumentations
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])


transform_val = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])




---

# --------------------------------------------

# 📘 **3 — Import Libraries & Setup Environment**

# --------------------------------------------

This step:

* Imports all required Python libraries
* Sets random seeds for reproducibility
* Confirms GPU availability

---

## ✅ **CELL 2 — Imports & Reproducibility**




In [ ]:
# Cell 2 — FIXED Dataset class (handles corrupted PNGs)

class BrainTumorDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_paths = sorted(glob.glob(os.path.join(image_dir, "*.png")))
        self.mask_paths  = sorted(glob.glob(os.path.join(mask_dir, "*.png")))

        # pair images and masks by filename
        mask_map = {os.path.basename(m): m for m in self.mask_paths}

        self.pairs = []
        for img in self.image_paths:
            name = os.path.basename(img)
            if name in mask_map:
                self.pairs.append((img, mask_map[name]))

        self.transform = transform

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, mask_path = self.pairs[idx]

        # ---- read image safely ----
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # corrupted file handling
        if img is None or mask is None:
            print("❌ Corrupted file skipped:", img_path)
            return self.__getitem__((idx + 1) % len(self.pairs))

        # convert to float32
        img = img.astype("float32")
        mask = mask.astype("float32")

        # normalize
        img /= 255.0
        mask = (mask > 127).astype("float32")

        # add channel dim
        img = np.expand_dims(img, axis=-1)
        mask = np.expand_dims(mask, axis=-1)

        # apply transforms
        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)
        else:
            img = torch.from_numpy(img).permute(2,0,1)
            mask = torch.from_numpy(mask).permute(2,0,1)

        return img, mask



---

# --------------------------------------------

# 📘 **4 — Dataset Paths / Kaggle Download Setup**

# --------------------------------------------

This step configures:

* Image directory
* Mask directory
* Optional Kaggle dataset download
* Validation that directories exist

---

## ✅ **CELL 3 — Set Dataset Paths**




In [ ]:
# Define image and mask directories

IMG_DIR = "/kaggle/input/brain-tumor-2021-dataset/kaggle/working/brats_images"
MASK_DIR = "/kaggle/input/brain-tumor-2021-dataset/kaggle/working/brats_masks"

print("IMG_DIR =", IMG_DIR)
print("MASK_DIR =", MASK_DIR)




---

# --------------------------------------------

# 📘 **5 — Create MRI Dataset Loader**

# --------------------------------------------

This step builds a **custom PyTorch Dataset class** that:

* Reads PNG/JPG images
* Pairs images and masks by filename
* Handles corrupted images
* Converts grayscale → 1-channel tensors
* Normalizes pixel values

---

## ✅ **CELL 4 — Dataset Class**



In [ ]:
import os, shutil

# Directories
SAVE_DIR = "/kaggle/working/"
TRAIN_IMG_DIR = os.path.join(SAVE_DIR, "train_images")
TRAIN_MASK_DIR = os.path.join(SAVE_DIR, "train_masks")
VAL_IMG_DIR = os.path.join(SAVE_DIR, "val_images")
VAL_MASK_DIR = os.path.join(SAVE_DIR, "val_masks")

# Create split directories
os.makedirs(TRAIN_IMG_DIR, exist_ok=True)
os.makedirs(TRAIN_MASK_DIR, exist_ok=True)
os.makedirs(VAL_IMG_DIR, exist_ok=True)
os.makedirs(VAL_MASK_DIR, exist_ok=True)

# Load dataset to get image-mask pairs
dataset = BrainTumorDataset(IMG_DIR, MASK_DIR, transform_train)

# Split
train_list, val_list = simple_split(dataset.pairs, val_ratio=0.15)

print("Total images:", len(dataset.pairs))
print("Train:", len(train_list), " Val:", len(val_list))

# Copy training images/masks
print("\nSaving Train Images...")
for img_path, mask_path in train_list:
    shutil.copy(img_path, TRAIN_IMG_DIR)
    shutil.copy(mask_path, TRAIN_MASK_DIR)

# Copy validation images/masks
print("\nSaving Val Images...")
for img_path, mask_path in val_list:
    shutil.copy(img_path, VAL_IMG_DIR)
    shutil.copy(mask_path, VAL_MASK_DIR)

print("\n✅ Split completed and saved in:")
print(TRAIN_IMG_DIR)
print(TRAIN_MASK_DIR)
print(VAL_IMG_DIR)
print(VAL_MASK_DIR)




---

# --------------------------------------------

# 📘 **6 — Data Augmentation & Loader Setup**

# --------------------------------------------

This step:

* Creates training augmentations
* Performs 85/15 train-validation split
* Prepares efficient PyTorch DataLoaders

---

## ✅ **CELL 5 — Transforms, Split & DataLoaders**




In [ ]:
import shutil

# Directories
base = "/kaggle/working/"

folders = [
    "train_images",
    "train_masks",
    "val_images",
    "val_masks"
]

# Create one ZIP per folder
for folder in folders:
    folder_path = base + folder
    zip_path = base + folder  # output same name
    print(f"Zipping → {zip_path}.zip ...")
    shutil.make_archive(zip_path, 'zip', folder_path)

print("\n✅ All zip files created successfully!")




---

# --------------------------------------------

# 📘 **7 — Define U-Net Model Architecture**

# --------------------------------------------

This section implements:

* Encoder
* Bottleneck
* Decoder
* Skip Connections
* Final Sigmoid Output

This U-Net is ideal for biomedical segmentation tasks.

---

## ✅ **CELL 6 — U-Net Model Definition**




In [ ]:
# Cell 4 — Training Loop

from tqdm import tqdm

def validate(model, loader, device):
    model.eval()
    dices = []

    with torch.no_grad():
        for imgs, masks in loader:
            imgs = imgs.to(device)
            masks = masks.to(device)

            preds = model(imgs)
            preds_bin = (preds > 0.5).float()

            for p, t in zip(preds_bin, masks):
                dices.append(dice_coef(p, t).item())

    return np.mean(dices) if len(dices) > 0 else 0.0


EPOCHS = 10        # you can increase to 30 later
best_dice = 0.0

for epoch in range(1, EPOCHS + 1):
    model.train()
    train_losses = []

    loop = tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch}/{EPOCHS}")

    for imgs, masks in loop:
        imgs = imgs.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, masks)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        loop.set_postfix(loss=loss.item())

    # ---- Validation ----
    val_dice = validate(model, val_loader, device)

    print(f"\nEpoch {epoch}: TrainLoss={np.mean(train_losses):.4f}  ValDice={val_dice:.4f}")

    # ---- Save best model ----
    if val_dice > best_dice:
        best_dice = val_dice
        torch.save(model.state_dict(), "/kaggle/working/unet_brats21_best.pth")
        print(f"✅ Saved new best model (Dice={best_dice:.4f})")

print("\n🎯 Training Finished!")
print("Best Validation Dice:", best_dice)


#Colab Code

In [ ]:
# =========================================================
# 1️⃣ Install Required Libraries
# =========================================================
!pip install flask pyngrok torch torchvision pillow opencv-python numpy albumentations h5py tqdm scikit-learn matplotlib
!mkdir -p templates static uploads results models

print("✅ Installation complete!")




---

# --------------------------------------------

# 📘 **11 — Build Flask Backend (app.py)**

# --------------------------------------------

This step:

* Loads trained U-Net
* Accepts image/H5 uploads
* Converts H5 → PNG
* Runs segmentation
* Saves results in `results/` directory
* Exposes web routes for frontend

---

## ✅ **CELL 10 — Create app.py**




In [ ]:
%%writefile app.py
from flask import Flask, render_template, request, send_from_directory
import os, torch, cv2, numpy as np, h5py
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch import nn

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['RESULT_FOLDER'] = 'results'
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs(app.config['RESULT_FOLDER'], exist_ok=True)

# ===============================
# U-Net Definition
# ===============================
class DoubleConv(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )
    def forward(self,x): return self.seq(x)

class UNet(nn.Module):
    def __init__(self,in_ch=1,out_ch=1,base=32):
        super().__init__()
        self.d1=DoubleConv(in_ch,base)
        self.p1=nn.MaxPool2d(2)
        self.d2=DoubleConv(base,base*2)
        self.p2=nn.MaxPool2d(2)
        self.d3=DoubleConv(base*2,base*4)
        self.p3=nn.MaxPool2d(2)
        self.d4=DoubleConv(base*4,base*8)
        self.u1=nn.ConvTranspose2d(base*8,base*4,2,stride=2)
        self.c1=DoubleConv(base*8,base*4)
        self.u2=nn.ConvTranspose2d(base*4,base*2,2,stride=2)
        self.c2=DoubleConv(base*4,base*2)
        self.u3=nn.ConvTranspose2d(base*2,base,2,stride=2)
        self.c3=DoubleConv(base*2,base)
        self.outc=nn.Conv2d(base,out_ch,1)
    def forward(self,x):
        c1=self.d1(x); c2=self.d2(self.p1(c1)); c3=self.d3(self.p2(c2)); c4=self.d4(self.p3(c3))
        x=self.c1(torch.cat([self.u1(c4),c3],1))
        x=self.c2(torch.cat([self.u2(x),c2],1))
        x=self.c3(torch.cat([self.u3(x),c1],1))
        return torch.sigmoid(self.outc(x))

# ===============================
# Load model
# ===============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model_path = 'models/unet_brats_best.pth'

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print("✅ Loaded trained UNet model")
else:
    print("⚠️ No trained model found. Please train and save to 'models/'")

# ===============================
# Preprocessing
# ===============================
transform = A.Compose([
    A.Resize(256,256),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

def process_input(filepath):
    """Handle both .png and .h5 files"""
    ext = os.path.splitext(filepath)[-1].lower()
    if ext == '.h5':
        with h5py.File(filepath, 'r') as f:
            img = f['image'][()][...,0]  # take first MRI channel
            mask = np.max(f['mask'][()], axis=-1)
        img_n = ((img - img.min())/(img.max()-img.min()+1e-6)*255).astype(np.uint8)
        converted = filepath.replace('.h5','.png')
        cv2.imwrite(converted, img_n)
        return converted
    else:
        return filepath

def segment_brain_tumor(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (256,256))
    img_norm = img.astype(np.float32)/255.0
    img_norm = np.expand_dims(img_norm, axis=2)
    t = transform(image=img_norm)
    x = t['image'].unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model(x).cpu().numpy()[0,0]
    mask = (pred>0.5).astype(np.uint8)*255
    color_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    overlay = color_img.copy()
    overlay[mask>0] = (0,255,0)
    blended = cv2.addWeighted(color_img, 0.7, overlay, 0.3, 0)
    return blended, mask

# ===============================
# Flask routes
# ===============================
@app.route("/", methods=["GET", "POST"])
def home():
    original, segmented, mask_file = None, None, None
    if request.method == "POST":
        f = request.files['image']
        if not f:
            return render_template("index.html", error="Please upload a file.")
        filename = f.filename
        upload_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        f.save(upload_path)
        print(f"📁 Uploaded: {filename}")

        # handle .h5 conversion if needed
        processed = process_input(upload_path)

        seg_out = os.path.join(app.config['RESULT_FOLDER'], f"seg_{os.path.basename(processed)}")
        mask_out = os.path.join(app.config['RESULT_FOLDER'], f"mask_{os.path.basename(processed)}")

        result, mask = segment_brain_tumor(processed)
        cv2.imwrite(seg_out, result)
        cv2.imwrite(mask_out, mask)

        original = os.path.basename(processed)
        segmented = os.path.basename(seg_out)
        mask_file = os.path.basename(mask_out)

    return render_template("index.html", original=original, segmented=segmented, mask_file=mask_file)

@app.route("/uploads/<filename>")
def uploaded(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

@app.route("/results/<filename>")
def results(filename):
    return send_from_directory(app.config['RESULT_FOLDER'], filename)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000, debug=False)




---

# --------------------------------------------

# 📘 **12 — Create Frontend UI (HTML Template)**

# --------------------------------------------

This step builds:

* Clean upload interface
* Result display layout
* Error handling messages

This is placed in **templates/index.html**

---

## ✅ **CELL 11 — Create index.html**




In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html>
<head>
    <title>🧠 Brain Tumour Detection (H5 + PNG)</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<div class="container">
    <h1>🧠 Brain Tumour Identification using Deep Learning</h1>
    <p class="subtitle">Upload MRI (.h5 or .png) → Detect & Highlight tumour regions</p>

    <form method="post" enctype="multipart/form-data" class="upload-form">
        <input type="file" name="image" accept=".h5,image/*" required>
        <button type="submit">🔍 Identify Tumour</button>
    </form>

    {% if original %}
    <div class="results">
        <div class="image-box">
            <h3>Original MRI</h3>
            <img src="{{ url_for('uploads', filename=original) }}" width="300">
        </div>
        <div class="image-box">
            <h3>Segmented Result</h3>
            <img src="{{ url_for('results', filename=segmented) }}" width="300">
        </div>
        <div class="image-box">
            <h3>Tumour Mask</h3>
            <img src="{{ url_for('results', filename=mask_file) }}" width="300">
        </div>
    </div>
    {% endif %}
</div>
</body>
</html>




---

# --------------------------------------------

# 📘 **13 — Add Styling (CSS)**

# --------------------------------------------

This step:

* Designs color theme
* Styles upload container & buttons
* Formats results section

This is saved in **static/style.css**

---

## ✅ **CELL 12 — Create style.css**



In [ ]:
%%writefile static/style.css
body {
    font-family: 'Segoe UI', sans-serif;
    background: linear-gradient(135deg, #4b6cb7, #182848);
    color: white;
    text-align: center;
    padding: 40px;
}
h1 { font-size: 2.5em; margin-bottom: 10px; }
.subtitle { opacity: 0.8; margin-bottom: 20px; }
.upload-form {
    margin: 20px auto;
    background: rgba(255,255,255,0.1);
    padding: 25px;
    border-radius: 15px;
    width: 400px;
}
button {
    padding: 12px 20px;
    background: #22c55e;
    border: none;
    border-radius: 10px;
    font-size: 1em;
    color: white;
    cursor: pointer;
}
.results {
    margin-top: 30px;
    display: flex;
    justify-content: center;
    gap: 20px;
}
.image-box {
    background: rgba(255,255,255,0.1);
    padding: 15px;
    border-radius: 10px;
}
img { border-radius: 10px; }
.error {
    background: rgba(239, 68, 68, 0.3);
    border: 2px solid #ef4444;
    padding: 10px;
    margin: 20px auto;
    width: 400px;
    border-radius: 10px;
}




---

# 📘 **5 — Run Flask App + ngrok Deployment**

This step:

* Stops existing processes
* Starts Flask backend
* Opens ngrok HTTPS public URL

---

# 📘 **Authenticate ngrok (IMPORTANT)**

ngrok provides a **public HTTPS URL**.

Your token is removed for safety.

---

### 🔑 **How to Use ngrok**

#### **Step 1 — Get your Token**

👉 [https://dashboard.ngrok.com/get-started/your-authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)

#### **Step 2 — Add Token in Notebook**

```python
#from pyngrok import ngrok, conf
#conf.get_default().auth_token = "YOUR_TOKEN_HERE"
```

#### **Step 3 — Start Tunnel**

```python
#public_url = ngrok.connect(8000)
#print("🌍 Public URL:", public_url)
```

A public HTTPS link will appear — you can open your app from anywhere.

---

## ✅ **CELL 5: Run Server & ngrok**



In [ ]:
!pkill -f flask || echo "No Flask running"
!pkill -f ngrok || echo "No ngrok running"
!nohup python app.py > flask.log 2>&1 &

from pyngrok import ngrok, conf
conf.get_default().auth_token = "PASTE_YOUR_TOKEN_HERE"
public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)
!sleep 3 && tail -n 10 flask.log


In [ ]:


---

# --------------------------------------------

# 📘 **16 — Notebook Completed Successfully**

# --------------------------------------------

🎉 Your **Brain Tumor Segmentation System** is now fully implemented!

You can now:

* Upload MRI image or H5 file
* Perform segmentation
* View tumor overlay
* Deploy publicly using ngrok
* Demonstrate in viva/exam/company interview

---


